# GeoHarness v4.0: RAPIDS cuML / CPU Fallback Offset Decoder Training

이 노트북은 `GeoHarness` 프로젝트에서 수집한 `ml_dataset.csv` 파일과 `vworld_anchors.csv` 파일을 사용하여 Google WGS84 좌표 ↔ Naver KATECH/TM128 좌표 간의 **비선형 공간 왜곡 오프셋(Offset)** 을 학습합니다.

### 👉 팀원 가이드 (중요)
1. **Colab GPU / 로컬 CPU 자동 전환**: 이 코드는 GPU(RAPIDS cuML)가 감지되면 초고속으로 학습하고, 없으면 자동으로 Scikit-Learn(CPU)으로 돌아갑니다. 현재 5,000건 미만 데이터이므로 로컬 CPU로 돌려도 수 초 내에 끝납니다.
2. **필수 입력 파일**: 프로젝트의 `data/` 폴더 안에 `ml_dataset.csv`와 `vworld_anchors.csv`가 있어야 합니다.
3. **출력 파일**: 실행이 끝나면 `src/models/decoder.pkl` 파일이 생성됩니다. 이 파일이 GeoHarness API 서버에서 사용됩니다.

In [ ]:
# 1. 환경 설정 및 라이브러리 설치
!pip install scikit-learn pandas pyproj

In [ ]:
import os
import math
import csv
import logging
from pathlib import Path

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("RAPIDSTrainer")

# 프로젝트 루트 찾기
current_dir = Path.cwd()
while current_dir.name != 'GeoHarness' and current_dir.parent != current_dir:
    current_dir = current_dir.parent

if current_dir.name != 'GeoHarness':
    current_dir = Path.cwd()  # Colab root

print(f"Project root determined as: {current_dir}")

In [ ]:
# GPU (cuML) / CPU (sklearn) 자동 감지
try:
    import cudf
    import cuml
    from cuml.ensemble import RandomForestRegressor as RF
    from cuml.metrics import mean_squared_error
    GPU_AVAILABLE = True
    print("✅ NVIDIA RAPIDS (cuML/cuDF) detected — GPU mode")
except ImportError:
    import pandas as cudf
    from sklearn.ensemble import RandomForestRegressor as RF
    from sklearn.metrics import mean_squared_error
    GPU_AVAILABLE = False
    print("⚠️ RAPIDS not available — falling back to sklearn (CPU)")

try:
    import joblib
except ImportError:
    import pickle as joblib

In [ ]:
# 유틸리티 함수 (거리 및 방위각 계산)
def haversine_distance(lat1, lng1, lat2, lng2):
    R = 6371000
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlam = math.radians(lng2 - lng1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlam / 2) ** 2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))

def bearing(lat1, lng1, lat2, lng2):
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dlam = math.radians(lng2 - lng1)
    x = math.sin(dlam) * math.cos(phi2)
    y = math.cos(phi1) * math.sin(phi2) - math.sin(phi1) * math.cos(phi2) * math.cos(dlam)
    return (math.degrees(math.atan2(x, y)) + 360) % 360

def compute_anchor_features(g_lat, g_lng, anchors):
    min_dist = float("inf")
    min_bearing = 0
    for i, a in enumerate(anchors):
        d = haversine_distance(g_lat, g_lng, a["lat"], a["lng"])
        if d < min_dist:
            min_dist = d
            min_bearing = bearing(g_lat, g_lng, a["lat"], a["lng"])
    return min_dist, min_bearing

In [ ]:
# 2. 데이터 및 VWorld 안커 로드
dataset_path = current_dir / "data" / "ml_dataset.csv"
anchors_path = current_dir / "data" / "vworld_anchors.csv"

if not dataset_path.exists():
    raise FileNotFoundError(f"❌ 오류: {dataset_path} 파일을 찾을 수 없습니다.")

df = cudf.read_csv(dataset_path)
print(f"Dataset loaded: {len(df)} rows")

anchors = []
if anchors_path.exists():
    with open(anchors_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            anchors.append({
                "name": row["anchor_name"],
                "lat": float(row["vw_lat"]),
                "lng": float(row["vw_lng"]),
            })
    print(f"Loaded {len(anchors)} VWorld anchors")
else:
    print("⚠️ vworld_anchors.csv not found, proceeding without anchor features.")

In [ ]:
# 3. 타겟 및 피쳐 생성 (델타 변환)
# (매우 중요) 원본 좌표계 자체가 아니라, '구글 좌표가 네이버 좌표와 얼마나 다른가(delta)'를 학습합니다.
df["delta_x"] = df["n_mapx"].astype(float) - df["g_lng"]
df["delta_y"] = df["n_mapy"].astype(float) - df["g_lat"]

if anchors:
    anchor_dists = []
    anchor_bearings = []
    df_pd = df.to_pandas() if GPU_AVAILABLE else df
    
    for _, row in df_pd.iterrows():
        dist, bear = compute_anchor_features(float(row["g_lat"]), float(row["g_lng"]), anchors)
        anchor_dists.append(dist)
        anchor_bearings.append(bear)
        
    if GPU_AVAILABLE:
        import cudf as real_cudf
        df["anchor_dist"] = real_cudf.Series(anchor_dists)
        df["anchor_bearing"] = real_cudf.Series(anchor_bearings)
    else:
        df["anchor_dist"] = anchor_dists
        df["anchor_bearing"] = anchor_bearings
        
    feature_cols = ["g_lat", "g_lng", "anchor_dist", "anchor_bearing"]
else:
    feature_cols = ["g_lat", "g_lng"]

print(f"설정된 Features: {feature_cols}")

In [ ]:
# 4. 모델 훈련 (RandomForest)
test_ratio = 0.1
n_test = max(1, int(len(df) * test_ratio))
n_train = len(df) - n_test

X = df[feature_cols]
y = df[["delta_x", "delta_y"]]

X_train, X_test = X.iloc[:n_train], X.iloc[n_train:]
y_train, y_test = y.iloc[:n_train], y.iloc[n_train:]

print(f"Training split: {n_train} train / {n_test} test")

model_x = RF(n_estimators=200, random_state=42)
model_y = RF(n_estimators=200, random_state=42)

print("Training model_x (delta_x)...")
model_x.fit(X_train, y_train["delta_x"])
print("Training model_y (delta_y)...")
model_y.fit(X_train, y_train["delta_y"])
print("Training complete! ✅")

In [ ]:
# 5. 성능 평가
pred_x = model_x.predict(X_test)
pred_y = model_y.predict(X_test)

if GPU_AVAILABLE:
    mse_x = mean_squared_error(y_test["delta_x"], pred_x).item()
    mse_y = mean_squared_error(y_test["delta_y"], pred_y).item()
else:
    mse_x = mean_squared_error(y_test["delta_x"], pred_x)
    mse_y = mean_squared_error(y_test["delta_y"], pred_y)

rmse_x = mse_x ** 0.5
rmse_y = mse_y ** 0.5

print(f"--- 오프셋 예측 오차 (RMSE) ---")
print(f"X축 에러: {rmse_x:.6f}")
print(f"Y축 에러: {rmse_y:.6f}")

In [ ]:
# 6. 모델 번들 저장 (inference.py 가 읽을 수 있는 형식으로 Export)
output_path = current_dir / "src" / "models" / "decoder.pkl"
output_path.parent.mkdir(parents=True, exist_ok=True)

model_bundle = {
    "model_x": model_x,
    "model_y": model_y,
    "feature_cols": feature_cols,
    "anchors": anchors,
    "rmse_x": rmse_x,
    "rmse_y": rmse_y,
    "gpu_trained": GPU_AVAILABLE,
    "n_samples": len(df),
}

try:
    joblib.dump(model_bundle, output_path)
    print(f"🎉 성공: 모델 번들이 저장되었습니다 -> {output_path}")
    print("이 노트북의 작업이 모두 완료되었습니다. 이제 API 서버에서 ML 모델을 로드할 수 있습니다.")
except Exception as e:
    print(f"저장 실패: {e}")
    joblib.dump(model_bundle, 'decoder.pkl')
    print("로컬 디렉토리에 decoder.pkl 로 대신 저장했습니다.")